In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy
import scipy
import math
import matplotlib as mpl
import matplotlib.dates as mpd
import pylab as plt
import datetime as dtm
import sys
import os
#import socket
#import lmod
# lmod.load('system')
# lmod.load('texlive')
# lmod.
#
# TODO: phase out unreferenced hpc_lib calls...
import hpc_lib

/home/users/myoder96/Codes/HPC_analytics/hpc_lib.py:2665: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def process_sacct_row(rw, delim='\t', headers=None, types_dict={}, RH={}):
/home/users/myoder96/Codes/HPC_analytics/hpc_lib.py:2680: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def get_modulus_stats(X,Y, a_mult=1., a_mod=1., qs=numpy.array([.5, .75, .95])):


In [3]:
n_cpus = 4
#print('** epoch: {}'.format(mpd.get_epoch()))
if 'SLURM_CPUS_PER_TASK' in os.environ.keys():
    n_cpus = int(os.environ['SLURM_CPUS_PER_TASK'])
#
print(f'*** n_cpus: {n_cpus}')


*** n_cpus: 8


In [33]:
SHP = hpc_lib.SH_PART_obj()

/home/users/myoder96/Codes/HPC_analytics/hpc_lib.py:2669: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  #  for now, let's assume that the map() (or whatever) will consolidate and (pseudo-)vectorize. Maybe this should be a Process() object....
/home/users/myoder96/Codes/HPC_analytics/hpc_lib.py:2684: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  #  in some modulo

** DEBUG:  [('partition_name', 'S7'), ('partition_public', 'S3'), ('nodes_idle', 'i8'), ('nodes_total', 'i8'), ('cpu_cores_idle', 'i8'), ('cpu_cores_total', 'i8'), ('cpu_cores_queued', 'i8'), ('gpus_idle', 'i8'), ('gpus_total', 'i8'), ('gpus_queued', 'i8'), ('job_runtime_default', 'S2'), ('job_runtime_maximum', 'S2'), ('mem/core_default', 'S3'), ('mem/core_maximum', 'S4'), ('per-node_cores', 'S6'), ('per-node_mem(gb)', 'S8'), ('per-node_gpus', 'S4')]


In [31]:
tp = type(7)
print('tp: ', tp)

tp:  <class 'int'>


In [32]:
print(numpy.dtype(7))

TypeError: Cannot interpret '7' as a data type

In [22]:
print('** ', SHP.columns)
print('** ', SHP.SP)

/home/users/myoder96/Codes/HPC_analytics/hpc_lib.py:2661: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  [('q_{}'.format(q), '>f8') for q in qs])
/home/users/myoder96/Codes/HPC_analytics/hpc_lib.py:2676: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  return [None if vl=='' else types_dict.get(col,str)(vl)


**  ['partition_name', 'partition_public', 'nodes_idle', 'nodes_total', 'cpu_cores_idle', 'cpu_cores_total', 'cpu_cores_queued', 'gpus_idle', 'gpus_total', 'gpus_queued', 'job_runtime_default', 'job_runtime_maximum', 'mem/core_default', 'mem/core_maximum', 'per-node_cores', 'per-node_mem(gb)', 'per-node_gpus']
**  [['normal*', 'yes', 0, 179, 124, 4820, 2318, 0, 0, 6, '2h', '7d', '6gb', '8gb', '20-24', '128-191', '0'], ['bigmem', 'yes', 1, 9, 304, 504, 81, 0, 0, 0, '2h', '1d', '8gb', '64gb', '24-128', '384-4096', '0'], ['gpu', 'yes', 0, 26, 140, 748, 538, 24, 104, 109, '1h', '2d', '7gb', '12gb', '20-32', '191-256', '3-4'], ['dev', 'yes', 0, 4, 85, 104, 0, 62, 64, 0, '1h', '2h', '6gb', '8gb', '20-32', '128-256', '0-32'], ['hns', 'no', 0, 105, 1326, 3728, 3847, 0, 0, 0, '2h', '7d', '6gb', '25gb', '20-128', '128-1024', '0'], ['serc', 'no', 1, 233, 609, 9104, 12331, 52, 92, 73, '2h', '7d', '8gb', '16gb', '24-128', '191-1024', '0-8'], ['owners', 'no', 4, 1467, 3033, 46952, 630658, 619, 636, 

In [ ]:
for rw in SHP.SP:
    print('** ', rw.split())

In [17]:
print('** ', len(b'abcd'))

**  4


In [ ]:
print('** ', SHP.sh_output_raw)

In [ ]:
sp_data = SHP.sh_output_raw
print('*** ', sp_data[0].split('|') )

In [ ]:
col_cats = [s.strip() for s in sp_data[0].split('|') if not s.strip()=='']
print(f'** col_cats: {col_cats}')

In [ ]:
col_groups = [[s.strip() for s in x.split()] for x in sp_data[1].split('|') if not x.strip()=='']
print('** ', col_groups)

In [ ]:
cols = [f'{s1}_{s2}'.strip().replace(' ', '_') for s1,S in zip(col_cats, col_groups) for s2 in S]
print(' ***\n', cols)